In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [30]:
from datasets import load_dataset
dataset = load_dataset("scherrmann/financial_phrasebank_75agree_german")
german_df_train = pd.DataFrame(dataset['train'])
german_df_test = pd.DataFrame(dataset['test'])
german_df_eval = pd.DataFrame(dataset['validation'])
german_df = pd.concat([german_df_train, german_df_test, german_df_eval])
german_df.shape[0]

3453

In [31]:
import kagglehub

path = kagglehub.dataset_download("arcticgiant/french-financial-news")
french_df = pd.read_csv(f'{path}/FrenchNews.csv')
french_df.shape[0]

41543

In [32]:
dataset = load_dataset("NickyNicky/financial_phrasebank_traslate_En_Es")
spanish_df = pd.DataFrame(dataset['train'])
spanish_df.shape[0]

4830

In [33]:
german_df.isnull().sum()

sentence    0
label       0
dtype: int64

In [34]:
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
german_df['Sentiment'] = german_df['label'].map(sentiment_map)
german_df.rename(columns={'sentence': 'Sentence'}, inplace=True)
german_df = german_df[['Sentence', 'Sentiment']]
german_df = german_df.dropna()
german_df['Sentiment'].value_counts()

Sentiment
neutral     2146
positive     887
negative     420
Name: count, dtype: int64

In [35]:
german_df.shape[0]

3453

In [36]:
french_df['sentiment_title'] = french_df['Sentiment Vader Title'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df['sentiment_text'] = french_df['Sentiment Vader Text'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df['sentiment_url'] = french_df['Sentiment Vader TextURL'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
french_df_processed = french_df.drop(columns=['Sentiment Vader Title', 'Sentiment Vader Text', 'Sentiment Vader TextURL', 'Numero news', 'Numero page', 'Numero', 'Agency'])
french_df_processed = french_df_processed.rename(columns={"Titre": "title", "Contenu": "text", "URL": "url"})
french_df_processed = french_df_processed[['title','sentiment_title']]
french_df_processed = french_df_processed.rename(columns={"title": "Sentence", "sentiment_title": "Sentiment"})
french_df_processed['Sentiment'].value_counts()

Sentiment
neutral     18507
negative    13188
positive     9848
Name: count, dtype: int64

In [37]:
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
spanish_df['Sentiment'] = spanish_df['label'].map(sentiment_map)
spanish_df.rename(columns={'sentence_es': 'Sentence'}, inplace=True)
spanish_df = spanish_df[['Sentence', 'Sentiment']]
spanish_df = spanish_df.dropna()
spanish_df['Sentiment'].value_counts()

Sentiment
neutral     2870
positive    1359
negative     601
Name: count, dtype: int64

In [38]:
sample_size = 2000
french_df_processed = french_df_processed.sample(sample_size, random_state=42)
german_df = german_df.sample(sample_size, random_state=42)
spanish_df = spanish_df.sample(sample_size, random_state=42)
df = pd.concat([german_df, french_df_processed, spanish_df], ignore_index=True)
df['Sentiment'].value_counts()


Sentiment
neutral     3381
positive    1539
negative    1080
Name: count, dtype: int64

In [39]:
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Sentiment'])
train.shape, eval.shape

((4800, 2), (1200, 2))

In [40]:
train.to_csv("data/train_subset.csv", index=False)
eval.to_csv("data/eval_subset.csv", index=False)